In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Define el símbolo del ticker para el par de divisas (GBP/EUR)
ticker = 'GBPEUR=X'

# Obtener datos históricos
data = yf.download(ticker, start='2019-11-01', end='2020-10-31')

# Mantener solo la columna 'Close' (Cierre)
exchange_rate = data[['Close']]

# Restablecer el índice para que 'Date' (Fecha) sea una columna
exchange_rate = exchange_rate.reset_index()

# Renombrar las columnas a 'Date' (Fecha) y 'GBP to EUR' (GBP a EUR)
exchange_rate.columns = ['Date', 'GBP to EUR']

# Mostrar el DataFrame
exchange_rate


[*********************100%***********************]  1 of 1 completed


,Date,GBP to EUR
0,2019-11-01,1.15967
1,2019-11-04,1.15850
2,2019-11-05,1.15817
3,2019-11-06,1.16280
4,2019-11-07,1.16130
...,...,...
256,2020-10-26,1.10162
257,2020-10-27,1.10205
258,2020-10-28,1.10590
259,2020-10-29,1.10554


In [2]:
import pandas as pd

# Define the data
data2 = {
    'Date': ['12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', 
             '10-2020'],
    'Inflation (%)': [1.3, 1.8, 1.7, 1.5, 0.8, 0.5, 0.6, 1.0, 0.2, 0.5, 0.7]
}

#Fuente: https://www.statista.com/statistics/306648/inflation-rate-consumer-price-index-cpi-united-kingdom-uk/

# Create DataFrame
inflation = pd.DataFrame(data2)

# Display the DataFrame
inflation


,Date,Inflation (%)
0,12-2019,1.3
1,01-2020,1.8
2,02-2020,1.7
3,03-2020,1.5
4,04-2020,0.8
5,05-2020,0.5
6,06-2020,0.6
7,07-2020,1.0
8,08-2020,0.2
9,09-2020,0.5


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')

#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()
df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()
pd.options.display.max_columns = None
display(df_jo.head(5))
#df_jo = pd.merge(cr_cp, fe_cp, left_on='id', right_on='cash_request_id', how ="left") #inner       

,id_cr,id_fe,fe_cr_id,user_id,active,created_at,created_at_fe,amount,fee,stat_cr,stat_fe,transfer_type,type,to_receive_ini,to_receive_bank,to_reimbur,to_reimbur_cash,to_end,to_send,send_at,cr_received_date,money_back_date,reimbursement_date,paid_at,charge_moment,moderated_at,reason,category,from_date,to_date,recovery_status,updated_at,reco_creation,reco_last_update,updated_at_fe,Mes_created_at,cash_request_received_date,n_fees,n_backs,n_recovery,created_at_slot,created_at_slot_h,created_at_dow,needs_m_check,needs_m_check_recov
0,3,0,NaN,47,1,2019-11-19 13:57:53.511561,NaT,1.0,NaN,canceled,cr_regular,regular,nice,NaT,NaT,16 days 09:02:06.488439,NaT,NaT,NaT,NaT,NaT,NaT,2019-12-05 23:00:00.000000,NaT,NaN,2019-11-20 12:16:50.000000,NaN,NaN,NaT,NaT,nice,2020-12-14 10:25:57.710844,NaT,NaT,NaT,2019-11,NaT,0,0,0,7,13-Mañana,1,1,1
1,4,0,NaN,99001309,0,2019-12-09 14:47:35.190714,NaT,100.0,NaN,money_back,cr_regular,regular,nice,NaT,NaT,7 days 08:12:24.809286,NaT,0 days,NaT,NaT,NaT,2019-12-16 23:00:00,2019-12-16 23:00:00.000000,NaT,NaN,2019-12-09 14:58:28.000000,NaN,NaN,NaT,NaT,nice,2020-11-04 12:54:52.120014,NaT,NaT,NaT,2019-12,NaT,0,1,0,14,14-Tarde,0,0,0
2,5,0,NaN,804,1,2019-12-10 19:05:21.596873,NaT,100.0,NaN,rejected,cr_regular,regular,nice,NaT,NaT,29 days 23:59:59.999490,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:05:21.596363,NaT,NaN,2019-12-11 16:47:42.405646,NaN,NaN,NaT,NaT,nice,2019-12-11 16:47:42.407830,NaT,NaT,NaT,2019-12,NaT,0,0,0,14,19-Tarde,1,1,1
3,6,0,NaN,812,1,2019-12-10 19:05:48.921042,NaT,100.0,NaN,direct_debit_rejected,cr_regular,regular,nice,0 days 04:54:11.078958,NaT,57 days 03:54:11.078958,NaT,NaT,NaT,NaT,2019-12-11,NaT,2020-02-05 23:00:00.000000,NaT,NaN,2019-12-11 09:40:33.000000,NaN,NaN,NaT,NaT,pending,2020-12-18 22:47:47.993817,2020-03-20 14:12:53.208246,2020-12-18 22:47:48.008850,NaT,2019-12,2019-12-11,0,0,1,14,19-Tarde,1,1,1
4,7,0,NaN,191,1,2019-12-10 19:13:35.825460,NaT,100.0,NaN,rejected,cr_regular,regular,nice,NaT,NaT,29 days 23:59:59.999581,NaT,NaT,NaT,NaT,NaT,NaT,2020-01-09 19:13:35.825041,NaT,NaN,2019-12-11 09:46:59.777728,NaN,NaN,NaT,NaT,nice,2019-12-11 09:46:59.779773,NaT,NaT,NaT,2019-12,NaT,0,0,0,14,19-Tarde,1,1,1


In [3]:
# Define the data
data3 = {
    'Date': ['12-2019', '01-2020', '02-2020', '03-2020', '04-2020', '05-2020', '06-2020', '07-2020', '08-2020', '09-2020', 
             '10-2020'],
    'Unemployment rate (%)': [4.0, 4.1, 4.1, 4.2, 4.2, 4.2, 4.4, 4.7, 5.0, 5.2, 5.2]
}

#Fuente: https://www.oecd.org/en/data/indicators/unemployment-rate.html?oecdcontrol-59006032fa-var1=GBR&oecdcontrol-4c072e451c-var3=2020-10

# Create DataFrame
unemployment = pd.DataFrame(data3)

# Display the DataFrame
unemployment

,Date,Unemployment rate (%)
0,12-2019,4.0
1,01-2020,4.1
2,02-2020,4.1
3,03-2020,4.2
4,04-2020,4.2
5,05-2020,4.2
6,06-2020,4.4
7,07-2020,4.7
8,08-2020,5.0
9,09-2020,5.2


In [4]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Define el símbolo del ticker para BTC/GBP
ticker = 'BTC-GBP'

# Obtener datos históricos
data4 = yf.download(ticker, start='2019-11-01', end='2024-10-31')

# Mantener solo la columna 'Close' (Cierre)
btc_gbp_data = data4[['Close']]

# Restablecer el índice para que 'Date' (Fecha) sea una columna
btc_gbp_data = btc_gbp_data.reset_index()

# Renombrar las columnas a 'Date' (Fecha) y 'BTC to GBP' (BTC a GBP)
btc_gbp_data.columns = ['Date', 'BTC to GBP']

# Mostrar el DataFrame
btc_gbp_data


[*********************100%***********************]  1 of 1 completed


,Date,BTC to GBP
0,2019-11-01,7156.055176
1,2019-11-02,7205.004395
2,2019-11-03,7139.003418
3,2019-11-04,7302.200684
4,2019-11-05,7253.519531
...,...,...
1821,2024-10-26,51413.687500
1822,2024-10-27,51704.855469
1823,2024-10-28,52424.503906
1824,2024-10-29,53880.703125


In [5]:
# Combine the two DataFrames by the 'Date' column
combined_df_daily_exogens = pd.merge(exchange_rate, btc_gbp_data, on='Date', how='inner')

# Mostrar el DataFrame combinado
combined_df_daily_exogens


,Date,GBP to EUR,BTC to GBP
0,2019-11-01,1.15967,7156.055176
1,2019-11-04,1.15850,7302.200684
2,2019-11-05,1.15817,7253.519531
3,2019-11-06,1.16280,7280.433594
4,2019-11-07,1.16130,7228.910645
...,...,...,...
256,2020-10-26,1.10162,10046.040039
257,2020-10-27,1.10205,10478.137695
258,2020-10-28,1.10590,10217.204102
259,2020-10-29,1.10554,10391.864258


In [6]:
# Combine the two DataFrames by the 'Date' column
combined_df_monthly_exogens = pd.merge(unemployment, inflation, on='Date', how='inner')

# Mostrar el DataFrame combinado
combined_df_monthly_exogens

,Date,Unemployment rate (%),Inflation (%)
0,12-2019,4.0,1.3
1,01-2020,4.1,1.8
2,02-2020,4.1,1.7
3,03-2020,4.2,1.5
4,04-2020,4.2,0.8
5,05-2020,4.2,0.5
6,06-2020,4.4,0.6
7,07-2020,4.7,1.0
8,08-2020,5.0,0.2
9,09-2020,5.2,0.5
